# Introduction

This notebooks presents **something** in Framework to solve **A Problem**.

**Contents**
* [Imports](#Imports)
* [Dataset](#Dataset)
* [Model](#Model)

**References**
* [Deep Residual Learning for Image Recognition]() (2015) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

# Config

In [ ]:
dataset_location = '/home/user/datasets/catsdogs'

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True)  # init TF ...
config=tf.ConfigProto(gpu_options=gpu_options)  # w/o taking ...
with tf.Session(config=config): pass            # all GPU memory

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets

# Dataset

* read raw data
  * drop, convert, etc.
* preprocessing
  * encoding (one-hot)
  * normalise
  * split train/valid
  * split features/targets
* data loaders
* (optional) save processed data

# Preprocess (opt.)

* e.g. NLP preprocessing, 

# Model

* (optional) load processed data
  * push to GPU
* define model/loss
* callbacks
  * schedulers
  * tensorboard
  * checkpoints
* train loop
* evaluate

# Experiment 1 (opt.)

* if many experiments

# Testing

* any unit testing

Pick GPU if available

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Custom Model

In [ ]:
class MyModel(nn.Module):
    def __init__(self, n_in, n_hid, n_out):
        super().__init__()          # !
        self.fc1 = nn.Linear(in_features=n_in, out_features=n_hid)
        self.fc2 = nn.Linear(in_features=n_hid, out_features=n_out)
    
    def forward(self, x):           # [n_batch, n_in]
        x = self.fc1(x)             # [n_batch, n_hid]
        x = self.fc2(x)             # [n_batch, n_out]
        return x

Instantinate

In [ ]:
model = CharRNN(nb_layers, n_in, n_embed, n_hid, n_out, dropout)
model.to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Train Loop

In [ ]:
batch_size = 1000
hist = { 'epoch': 0,
         'train_loss':[], 'train_accuracy':[],
         'valid_loss':[], 'valid_accuracy':[]
       }

def train(hist, nb_epochs, train_dl, valid_dl):
    
    for _ in range(nb_epochs):
        epoch = hist['epoch']
        
        ### Train ###
        model.train()
        loss_sum, accuracy_sum = 0, 0
        for features, targets in train_dl:
            
            # Push to GPU
            features = features.to(device)
            targets = targets.to(device)
            
            # Optimize
            outputs = model(features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            with torch.no_grad():
                acc = accuracy(outputs, targets)
                loss_sum += loss.item()
                accuracy_sum += acc.item()
                hist['iter_epoch'].append( epoch )
                hist['iter_accuracy'].append( acc.item() )
                hist['iter_loss'].append( loss.item() )
        
        hist['train_loss'].append( loss_sum / len(train_dl) )
        hist['train_accuracy'].append( accuracy_sum / len(train_dl) )
        
        ### Evaluate ###
        model.eval()
        loss_sum, accuracy_sum = 0, 0
        for features, targets in valid_dl:
            
            # Push to GPU
            features = features.to(device)
            targets = targets.to(device)
            
            # Eval
            with torch.no_grad():
                outputs = model(features)
                loss = criterion(outputs, targets)
                acc = accuracy(outputs, targets)
                
                loss_sum += loss.item()
                accuracy_sum += acc.item()
        
        hist['valid_loss'].append( loss_sum / len(train_dl) )
        hist['valid_accuracy'].append( accuracy_sum / len(train_dl) )
        
        ### Print Summary ###
        if epoch == 0:
            print('      (time )   ep             Loss (t/v)                Acc (t/v)')
        print(f'Epoch ({epoch_time_interval:4.2}s): {epoch:3}'
              f'    {hist["train_loss"][-1]:6.4f} / {hist["valid_loss"][-1]:6.4f}'
              f'    {hist["train_acc"][-1]:6.4f} / {hist["valid_acc"][-1]:6.4f}')

Keras generator

In [ ]:
class CustomSequence(tf.keras.utils.Sequence):

    def __init__(self, features, targets, batch_size, shuffle=False):
        self.features = features
        self.targets = targets
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.features) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size :
                                     (idx+1) * self.batch_size]
        batch_features = self.features[batch_i]
        batch_targets = self.targets[batch_i]

        return batch_features, batch_targets

    def on_epoch_end(self):
        self.indices = np.arange(len(self.data_x))
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
class CallbackPlot(tf.keras.callbacks.Callback):
    def __init__(self, train_images, valid_images):
        self.train_images = train_images
        self.valid_images = valid_images
    
    def on_train_begin(self, logs={}):
        pass

    def on_epoch_end(self, batch, logs={}):
        _, iw, ih, _ = self.train_images.shape
        
        predictions = model.predict(self.train_images)
        plot_images([5, 10, 15, 20, 25, 30],
                    undo_preprocess_images(self.train_images),
                    undo_preprocess_keypts(predictions, iw))
        
        predictions = model.predict(self.valid_images)
        plot_images([5, 10, 15, 20, 25, 30],
                    undo_preprocess_images(self.valid_images),
                    undo_preprocess_keypts(predictions, iw))

In [ ]:
hist = model.fit_generator(train_generator,
                           epochs=20,
                           validation_data=test_generator)